# SPOTIFY

#### Connect and Get all tracks

In [3]:
import spotipy
from pprint import pprint

## LOAD ENVIRONMENT VARIABLES : 
from dotenv import load_dotenv
import os
load_dotenv()


spotify = spotipy.Spotify(auth_manager=spotipy.SpotifyOAuth())
me = spotify.me()
pprint(me)



{'display_name': 'Thealamenthe',
 'external_urls': {'spotify': 'https://open.spotify.com/user/jbelguiz'},
 'followers': {'href': None, 'total': 17},
 'href': 'https://api.spotify.com/v1/users/jbelguiz',
 'id': 'jbelguiz',
 'images': [],
 'type': 'user',
 'uri': 'spotify:user:jbelguiz'}


In [4]:
# Shows a user's saved tracks (need to be authenticated via oauth) 
# 
# MAIN FUNCTION IN USE !
# add track id for every track 

import spotipy
from spotipy.oauth2 import SpotifyOAuth

scope = 'user-library-read'

tracklist = []
artistlist = []
albumlist = []
trackid = []

def show_tracks(results):
    for item in results['items']:
        track = item['track']
        print("%32.32s %s -id %s --album : %s" % (track['artists'][0]['name'], track['name'],track['id'],track['album']['name']))
        tracklist.append(track['name'])
        artistlist.append(track['artists'][0]['name'])
        albumlist.append(track['album']['name'])
        trackid.append(track['id'])


sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

results = sp.current_user_saved_tracks()
show_tracks(results)

while results['next']:
    results = sp.next(results)
    show_tracks(results)

Opening in existing browser session.
                  Punkin Machine I Need You Tonight -id 12SY1IDzol5dg1yBTUoKYx --album : I Need You Tonight / Tonight
                Childish Gambino Redbone -id 2Uvy6SkNqxvnH1W68dymxG --album : "Awaken, My Love!"
                 Mayer Hawthorne Henny & Gingerale - Bonus Track -id 22WRYBnzXfM9g7uNd5w79q --album : How Do You Do
                       Lord Echo Woah! Theres No Limit -id 6Rd2NhQnVeDBMUti7BiS90 --album : Harmonies
                     Jon Batiste Chopinesque -id 7hWuW5wngkJxrHku0TMwDC --album : Hollywood Africans
                    Nancy Wilson I Wish You Love -id 7moJWeYfMTr7iWgSHvwF7Y --album : Something Wonderful
                     Kris Bowers Wake The Neighbors -id 2k7zIn3RLutXKNn8RMnlhU --album : Heroes + Misfits
                          Noname Shadow Man (feat. Phoelix, Smino & Saba) -id 1h2LHhmyAXi8tPNLi806JA --album : Telefone
                     Stacey Kent La Rua Madureira -id 3bSCmmrIRMYyl9BLH8zLMm --album : I Know I D

In [5]:
import pandas as pd
dictionary = {
    'Artists': artistlist, 
    'Tracks': tracklist, 
    'Albums': albumlist,
    'TrackID': trackid
    }
df_library_spotify = pd.DataFrame(dictionary)
df_library_spotify

,Artists,Tracks,Albums,TrackID
0,Punkin Machine,I Need You Tonight,I Need You Tonight / Tonight,12SY1IDzol5dg1yBTUoKYx
1,Childish Gambino,Redbone,"""Awaken, My Love!""",2Uvy6SkNqxvnH1W68dymxG
2,Mayer Hawthorne,Henny & Gingerale - Bonus Track,How Do You Do,22WRYBnzXfM9g7uNd5w79q
3,Lord Echo,Woah! Theres No Limit,Harmonies,6Rd2NhQnVeDBMUti7BiS90
4,Jon Batiste,Chopinesque,Hollywood Africans,7hWuW5wngkJxrHku0TMwDC
...,...,...,...,...
4552,S-Crew,Un homme change le monde,Destins liés,4YIeLqRqMTWRb3ydpHGTAu
4553,S-Crew,Plus pareil,Destins liés,56gPpeDBQkBhQaEsq7bRI0
4554,S-Crew,Bresson,Destins liés,2p21IPU7QhbTCHtpvoSyre
4555,S-Crew,Une étoile,Destins liés,1bAHsggx8whuj34dybkvb9


#### Get all playlist and their tracks

In [21]:
# playlist track id 
def trackID_from_playlistID (playlistID): 
    ''' Return a list of track ID for a given PlaylistID
    '''
    pl_id = 'spotify:playlist:'+playlistID
    offset = 0
    trackIDS=[]
    while True:
        response = sp.playlist_items(pl_id,
                                    offset=offset,
                                    fields='items.track.id,total',
                                    additional_types=['track'])
        
        if len(response['items']) == 0:
            break
        
        #pprint(response['items'])
        offset = offset + len(response['items'])
        #print(offset, "/", response['total'])

        for track in response['items']:
            trackIDS.append(track['track']['id'])

    if response['total']==len(trackIDS):
        return trackIDS
    else : 
        return None

# get playlist id : 

scope = 'playlist-read-private'
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

playlists = sp.current_user_playlists(limit=50)

playlist_name = []
playlist_id = []
for i, item in enumerate(playlists['items']):
    playlist_id.append(item['id'])
    playlist_name.append(item['name'])
    print("%d %s %s" % (i, item['name'], item['id']))

# get tracks from playlists

df_playlist = pd.DataFrame(columns=['PlaylistName','TrackID'])

for i in range(len(playlist_id)):
    # add one playlist
    df = pd.DataFrame(columns=['PlaylistName','TrackID'])
    df['TrackID'] = trackID_from_playlistID(playlist_id[i])
    df['PlaylistName'] = playlist_name[i]   
    df_playlist = pd.concat([df_playlist,df],ignore_index=True)

df_playlist

Opening in existing browser session.
0 Your Top Songs 2021 37i9dQZF1EUMDoJuT8yJsl
1 Your Top Songs 2020 37i9dQZF1EMe2T0EyvguLP
2 Italove Radio 37i9dQZF1E8NbdbX2F45JR
3 Rap instru 1dXfMuIRYsKGJQ8M0umDtL
4 Rap Workout 37i9dQZF1DX76t638V6CA8
5 Italia 1yST9FK3aQR3nz3wTVEswY
6 italian way 0BlDB5xOIN3sVZU8mxxxud
7 Your Top Songs 2018 37i9dQZF1EjoRhwNKmilGy
8 La Playlist de France Inter (officiel) 1IhrDzE8Z4fegSvS8uHmJM
9 Minimal Techno 5fIKyffSDYwk3Lsp1fu6Ix
10 So ->2020 3SLqjv6aiWJvQPU3AOiZaq
11 Liked from Radio 0WkVZnEDeEBmYODkNwInpM
12 Shs 0GJ4Jn4Aiey2cryR6Qknaq
13 Inspi 3fdk3ojDPHoo2goiKauFYZ
14 Instrumental Hip Hop / Chill Hip Hop / Jazzy 1dQQkNsgAcfAzO1KTvHOTR
15 Your Top Songs 2017 37i9dQZF1E9TRuLHu7N8Tt
16 Mes titres Shazam 1OTkYknLsOm6EA9KmvSfJ0
17 Romantic songs 20jq3psqbPdWpDBlrCGPP5
18 Spotify's Most Played All-Time [Updated Weekly] | Most Streamed | Top Played | 500Mil+ 2YRe7HRKNRvXdJBp9nXFza
19 Jazz 4CBhdnEFSnAFT0Cd8mfJ6L
20 Unexpected 3yE2LMbxKxT5DHOFvZowqY
21 Rap - Turn Up 7C

,PlaylistName,TrackID
0,Your Top Songs 2021,3JJL91ilRV6fXhKqu0FGXs
1,Your Top Songs 2021,0BiK5BbYNFLb88CCOxBFJe
2,Your Top Songs 2021,2goHCrsDJaflEwQ930ef0o
3,Your Top Songs 2021,4ydmav4vl7hebadtfZtMrg
4,Your Top Songs 2021,13HsOwrwTA5HgLuoaDHncP
...,...,...
5270,Sexy songs - R&B,2rWXpcM2lmlf2i7qCNaXHM
5271,Sexy songs - R&B,4LkLfPYc3yjOta7Gk8FDRB
5272,Sexy songs - R&B,42QRyJd5x6e2TFR1S41g2p
5273,Sexy songs - R&B,5MjAFPcmrR1DYCgZrXCN4d


In [71]:
# playlists infos / Alternative method
def show_tracks(results):
    for i, item in enumerate(results['items']):
        track = item['track']
        print(
            "   %d %32.32s %s - %s" %
            (i, track['artists'][0]['name'], track['name'],track['id']))


scope = 'playlist-read-private'
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

playlists = sp.current_user_playlists()
user_id = sp.me()['id']

for playlist in playlists['items']:
    if playlist['owner']['id'] == user_id:
        print()
        print(playlist['name'])
        print('  total tracks', playlist['tracks']['total'])

        results = sp.playlist(playlist['id'], fields="tracks,next")
        tracks = results['tracks']
        show_tracks(tracks)

        while tracks['next']:
            tracks = sp.next(tracks)
            show_tracks(tracks)


Italia
  total tracks 17
   0                   Ornella Vanoni La musica è finita - 7xysI1rMkivCZ21aaiSyCl
   1                   Auroro Borealo Gli occhi del mio ex - 01E82CAhGlE31BrnAWbGdX
   2                     Pino D'Angiò Ma quale idea - 50f0mt35cIwwu3Krk7nS7b
   3                   Ornella Vanoni Musica musica - 2j4sJrXXQG9dOspVijddt9
   4                     Rino Gaetano Ma il cielo è sempre più blu - 1L9m2l2lhXgQ3GwmmCx3OK
   5                        Jovanotti Un Raggio Di Sole - 7kaG6kUGXZhCy27g1JgL0S
   6                       Gino Paoli La gatta - 1f08sBI8nFXyQIU6FmOxSn
   7                    Enzo Siffredi She Babao - 5IUQCQ7J9prAJ6hnTnWnGx
   8                       DJ Gregory Canoa - 7yAqzJAp6RaOtiDL4MgKzx
   9               Riccardo Del Turco Cosa hai messo nel caffè? - 2BTvcx218RPaIXyH5Gz3xa
   10                     Donato Dozzy Parola - Rework - 0GQZojX9G6n6YADImDzYW4
   11                   Ornella Vanoni Ti voglio - 14dvepiFqD3a2lBwmVbhB3
   12                Vin

#### Select Playlists to keep 


In [23]:
df_playlist.PlaylistName.unique()

# to delete : 

to_delete=['Rap Workout','Italove Radio','Rap instru','Rap Workout','italian way','La Playlist de France Inter (officiel)', 'Minimal Techno',
    'So ->2020','Instrumental Hip Hop / Chill Hip Hop / Jazzy',"Spotify's Most Played All-Time [Updated Weekly] | Most Streamed | Top Played | 500Mil+"]

df_playlist_tokeep = df_playlist[~df_playlist['PlaylistName'].isin(to_delete)]
df_playlist_tokeep.reset_index(drop=True,inplace=True)
df_playlist_tokeep

,PlaylistName,TrackID
0,Your Top Songs 2021,3JJL91ilRV6fXhKqu0FGXs
1,Your Top Songs 2021,0BiK5BbYNFLb88CCOxBFJe
2,Your Top Songs 2021,2goHCrsDJaflEwQ930ef0o
3,Your Top Songs 2021,4ydmav4vl7hebadtfZtMrg
4,Your Top Songs 2021,13HsOwrwTA5HgLuoaDHncP
...,...,...
732,Sexy songs - R&B,2rWXpcM2lmlf2i7qCNaXHM
733,Sexy songs - R&B,4LkLfPYc3yjOta7Gk8FDRB
734,Sexy songs - R&B,42QRyJd5x6e2TFR1S41g2p
735,Sexy songs - R&B,5MjAFPcmrR1DYCgZrXCN4d


#### Select from playlists tracks to add to library


In [ ]:
# add all tracks from selected playlist to library 

In [24]:
# tracks ID with playlist name
df_playlist_tokeep

,PlaylistName,TrackID
0,Your Top Songs 2021,3JJL91ilRV6fXhKqu0FGXs
1,Your Top Songs 2021,0BiK5BbYNFLb88CCOxBFJe
2,Your Top Songs 2021,2goHCrsDJaflEwQ930ef0o
3,Your Top Songs 2021,4ydmav4vl7hebadtfZtMrg
4,Your Top Songs 2021,13HsOwrwTA5HgLuoaDHncP
...,...,...
732,Sexy songs - R&B,2rWXpcM2lmlf2i7qCNaXHM
733,Sexy songs - R&B,4LkLfPYc3yjOta7Gk8FDRB
734,Sexy songs - R&B,42QRyJd5x6e2TFR1S41g2p
735,Sexy songs - R&B,5MjAFPcmrR1DYCgZrXCN4d


In [25]:
#library
df_library_spotify

,Artists,Tracks,Albums,TrackID
0,Punkin Machine,I Need You Tonight,I Need You Tonight / Tonight,12SY1IDzol5dg1yBTUoKYx
1,Childish Gambino,Redbone,"""Awaken, My Love!""",2Uvy6SkNqxvnH1W68dymxG
2,Mayer Hawthorne,Henny & Gingerale - Bonus Track,How Do You Do,22WRYBnzXfM9g7uNd5w79q
3,Lord Echo,Woah! Theres No Limit,Harmonies,6Rd2NhQnVeDBMUti7BiS90
4,Jon Batiste,Chopinesque,Hollywood Africans,7hWuW5wngkJxrHku0TMwDC
...,...,...,...,...
4552,S-Crew,Un homme change le monde,Destins liés,4YIeLqRqMTWRb3ydpHGTAu
4553,S-Crew,Plus pareil,Destins liés,56gPpeDBQkBhQaEsq7bRI0
4554,S-Crew,Bresson,Destins liés,2p21IPU7QhbTCHtpvoSyre
4555,S-Crew,Une étoile,Destins liés,1bAHsggx8whuj34dybkvb9


# Deezer

launch the follwing in the console to get token:<br><br>
*python oauth_deezer.py --app-id 521242 --app-secret 1b2d32133792c353a555ff83c31ac6d8*<br>
*source .env*

<br><br>

In [6]:
import deezer
## LOAD ENVIRONMENT VARIABLES : 
from dotenv import load_dotenv
import os
load_dotenv()

client = deezer.Client(access_token=os.environ['DEEZER_TOKEN'],app_id=os.environ['DEEZER_APP_ID'],app_secret=os.environ['DEEZER_CLIENT_SECRET'])

In [7]:

Tracks_id = []
unmatched=[]
for index,row in df_library_spotify.iterrows():
    print(index)
    print(row.Artists)
    print(row.Tracks)
    print(row.Albums)

    if index in (128,1471,1901,4046) :
        unmatched.append([row.Artists,row.Tracks,row.Albums])
        print('unmatched\n')
        pass
    else:

        results = client.search(artist=row.Artists,track=row.Tracks,album=row.Albums)
        
        if len(results)>0:
            Tracks_id.append(results[0].id)
            print(' Track :'+ row.Tracks +' from '+row.Artists +' added\n')
        else :
            unmatched.append([row.Artists,row.Tracks,row.Albums])
            print('unmatched\n')

#results

0
Punkin Machine
I Need You Tonight
I Need You Tonight / Tonight
 Track :I Need You Tonight from Punkin Machine added

1
Childish Gambino
Redbone
"Awaken, My Love!"
 Track :Redbone from Childish Gambino added

2
Mayer Hawthorne
Henny & Gingerale - Bonus Track
How Do You Do
unmatched

3
Lord Echo
Woah! Theres No Limit
Harmonies
 Track :Woah! Theres No Limit from Lord Echo added

4
Jon Batiste
Chopinesque
Hollywood Africans
 Track :Chopinesque from Jon Batiste added

5
Nancy Wilson
I Wish You Love
Something Wonderful
 Track :I Wish You Love from Nancy Wilson added

6
Kris Bowers
Wake The Neighbors
Heroes + Misfits
 Track :Wake The Neighbors from Kris Bowers added

7
Noname
Shadow Man (feat. Phoelix, Smino & Saba)
Telefone
 Track :Shadow Man (feat. Phoelix, Smino & Saba) from Noname added

8
Stacey Kent
La Rua Madureira
I Know I Dream : The Orchestral Sessions (Deluxe Version)
 Track :La Rua Madureira from Stacey Kent added

9
Lord Echo
Thinking of You
Melodies
 Track :Thinking of You fro

In [8]:
# add every Trackid to the library if not already:

len(unmatched)

562

In [15]:
# with all unmatched launch new algorithm without the album name

Tracks_id_v2 = []
unmatched_v2 = []
song_album = 0

for index,track in enumerate(unmatched):

    artist_ = track[0]
    song_ = track[1]
    album_ = track[2]

    print(index)
    print(track[0])
    print(track[1])
    print(track[2])

    if  index in ([11,121,189,489]) :
        unmatched_v2.append([artist_,song_,album_])
        print('unmatched\n')
        pass
    else:
        results = client.search(artist=artist_,track=song_)
        if len(results)>0:
            Tracks_id_v2.append(results[0].id)
            print(' Track :'+ song_ +' from '+artist_ +' added\n')
        else :
            results = client.search(track=song_,album=album_)
            if len(results)>0:
                Tracks_id_v2.append(results[0].id)
                print(' Track :'+ song_ +' from '+artist_ +' added\n')
                song_album = song_album +1           
            else:
                unmatched_v2.append([artist_,song_,album_])
                print('unmatched\n')

len(unmatched_v2)

0
Mayer Hawthorne
Henny & Gingerale - Bonus Track
How Do You Do
unmatched

1
Adriano Celentano
L'Emozione Non Ha voce (Io Non So Parlar D'Amore)
Io Non So Parlar D'Amore
 Track :L'Emozione Non Ha voce (Io Non So Parlar D'Amore) from Adriano Celentano added

2
Guè
Smith & Wesson Freestyle (feat. Marracash)
Fastlife 4
unmatched

3
Daft Punk
Harder, Better, Faster, Stronger
Discovery
 Track :Harder, Better, Faster, Stronger from Daft Punk added

4
Netinho
Milla - Ao Vivo
Total - O Melhor De Netinho
 Track :Milla - Ao Vivo from Netinho added

5
Billie Eilish
Lo Vas A Olvidar (with ROSALÍA)
Lo Vas A Olvidar (with ROSALÍA)
unmatched

6
GoldLink
Sorrow, Tears And Blood
Sorrow, Tears And Blood
 Track :Sorrow, Tears And Blood from GoldLink added

7
Jacob Banks
Devil That I Know
Devil That I Know
 Track :Devil That I Know from Jacob Banks added

8
Johannes Brahms
Symphony No. 3 in F Major, Op. 90: III. Poco allegretto
Brahms: Symphonies 1-4
 Track :Symphony No. 3 in F Major, Op. 90: III. Poco al

235

In [42]:
# Get existing user tracks to do my user id : 3371978

r = client.get_user_tracks(3371978)

r

KeyboardInterrupt: 

TypeError: sequence item 0: expected str instance, int found

In [ ]:
show_tracks(results)

while results['next']:
    results = sp.next(results)
    show_tracks(results)

In [28]:
#Tracks_id.extend(Tracks_id_v2)
track_id_not_added = []
counter = 0
for track in Tracks_id :
    if client.add_user_track(track):
        counter = counter + 1
    else :
        track_id_not_added.append(track)

print(track_id_not_added)

DeezerErrorResponse: {'error': {'type': 'DataException', 'message': 'Track already exists', 'code': 801}}